# Operating Income Maximization for a Tea Producer Company

## Install and Import Libraries

In [ ]:
pip install pulp

In [186]:
import pulp

## Solve the Problem

1. **Initialize the model and set the known parameters.**

In [187]:
# Initialize the LP problem
prob = pulp.LpProblem("Maximize_Operating_Income", pulp.LpMaximize)

In [188]:
# Tea types and sizes
tea_types = ['GT', 'BT', 'WT', 'RT']
sizes = ['50g', '100g']
countries = ['Portugal', 'Spain', 'France', 'Italy', 'Germany', 'Poland']

# Define selling prices
selling_prices = {
    'GT':  {'50g': 6, '100g': 10},
    'BT':  {'50g': 7, '100g': 12},
    'WT':  {'50g': 9, '100g': 16},
    'RT':  {'50g': 8, '100g': 14}
}

# Define buying prices (per 100g)
buying_prices = {
    'GT':  3.5,
    'BT':  4.5,
    'WT':  5.5,
    'RT':  5
}

# Define distribution costs per country
distribution_costs = {
    'Portugal': 0.5,
    'Spain': 0.4,
    'France': 0.6,
    'Italy': 0.7,
    'Germany': 0.8,
    'Poland': 1.0
}

# Define demand forecast per country per product per size (in thousands)
demand_forecast = {
    'Portugal': {
        'GT': {'50g': 100, '100g': 80},
        'BT': {'50g': 90, '100g': 70},
        'WT': {'50g': 50, '100g': 40},
        'RT': {'50g': 60, '100g': 50}
    },
    'Spain': {
        'GT': {'50g': 150, '100g': 120},
        'BT': {'50g': 130, '100g': 110},
        'WT': {'50g': 80, '100g': 60},
        'RT': {'50g': 100, '100g': 90}
    },
    'France': {
        'GT': {'50g': 200, '100g': 170},
        'BT': {'50g': 180, '100g': 160},
        'WT': {'50g': 100, '100g': 90},
        'RT': {'50g': 120, '100g': 110}
    },
    'Italy': {
        'GT': {'50g': 130, '100g': 110},
        'BT': {'50g': 120, '100g': 100},
        'WT': {'50g': 70, '100g': 60},
        'RT': {'50g': 90, '100g': 80}
    },
    'Germany': {
        'GT': {'50g': 180, '100g': 160},
        'BT': {'50g': 170, '100g': 150},
        'WT': {'50g': 90, '100g': 80},
        'RT': {'50g': 110, '100g': 100}
    },
    'Poland': {
        'GT': {'50g': 120, '100g': 100},
        'BT': {'50g': 110, '100g': 90},
        'WT': {'50g': 60, '100g': 50},
        'RT': {'50g': 80, '100g': 70}
    }
}

# Raw material availability (grams)
available_tea = {
    'GT': 25000000,
    'BT': 30000000,
    'WT': 15000000,
    'RT': 20000000
}

2. **Define the decision variables.**

In [189]:
# Create decision variables
x = pulp.LpVariable.dicts("Boxes",
                          [(i, j, k) for i in tea_types for j in sizes for k in countries],
                          lowBound=0,
                          cat='Integer')

In [190]:
# Total production variable
T = pulp.LpVariable("Total_Production", lowBound=0, cat='Integer')

3. **Add constraints.**

  1. The total amount of tea produced should not exceed the total capacity of the factory.

In [191]:
# Total production constraint
prob += T == pulp.lpSum([x[(i,j,k)] for i in tea_types for j in sizes for k in countries]), "Total_Production_Definition"

# Total production capacity constraint
prob += T <= 2000000, "Total_Production_Capacity"

  2. The total amount of tea produced should not exceed the total raw material available for each type of tea.

In [192]:
# Raw material availability constraints
for i in tea_types:
    prob += pulp.lpSum([
        int(j[:-1]) * x[(i,j,k)] for j in sizes for k in countries
    ]) <= available_tea[i], f"Raw_Material_{i}"

  3. At least 10% of production should be allocated to each type of tea.

In [193]:
# Minimum production requirement constraints
for i in tea_types:
    prob += pulp.lpSum([x[(i,j,k)] for j in sizes for k in countries]) >= 0.10 * T, f"Min_Production_{i}"

  4. The total amount of tea produced should not exceed the demand forecast for each type of tea.

In [194]:
# Add demand constraints
for k in countries:
    for i in tea_types:
        for j in sizes:
            prob += x[(i,j,k)] <= demand_forecast[k][i][j] * 1000, f"demand_forecast_{i}_{j}_{k}"

4. **Calculate the profit per unit of each type of tea.**

In [195]:
# Calculating profit per box for each product
profit = {}
for i in tea_types:
    for j in sizes:
        selling_price = selling_prices[i][j]
        cost = buying_prices[i] * int(j[:-1]) / 100
        profit[(i, j)] = selling_price - cost

5. **Set the objective function.**

In [196]:
# Define production cost per box
PRODUCTION_COST = 1

# Add the objective function
prob += pulp.lpSum([(profit[(i,j)] - (distribution_costs[k] + PRODUCTION_COST)) * x[(i,j,k)] 
                     for i in tea_types for j in sizes for k in countries ])

6. **Solve the model.**

In [ ]:
# Solve the LP problem
prob.solve()

7. **Print the results.**

In [198]:
# Define fixed costs
MARKETING_COST = 400000
ANUAL_COST = 4000000

In [ ]:
# Output results
print("Status:", pulp.LpStatus[prob.status])
print()
print("Optimal Production and Distribution Plan:")
for v in prob.variables():
    if v.varValue > 0:
        print(f"{v.name} = {v.varValue}")
print()

# Calculate total operating income after subtracting fixed costs
total_contribution_margin = pulp.value(prob.objective)
total_operating_income = total_contribution_margin - MARKETING_COST - ANUAL_COST

print("Total Contribution Margin (before fixed costs) = ${0:,.2f}".format(total_contribution_margin))
print("Total Operating Income (after fixed costs) = ${0:,.2f}".format(total_operating_income))

We notice that the Production Capacity limit has not been reached, so we verify that all the raw materials have been used

In [ ]:
# Calculate and print raw material usage
print()
print("Raw Material Usage:")
for i in tea_types:
    total_grams = 0
    for j in sizes:
        for k in countries:
            var_name = f"Boxes_('{i}',_'{j}',_'{k}')"
            # Check if the variable exists
            if var_name in prob.variablesDict():
                var_value = prob.variablesDict()[var_name].varValue
                if var_value and var_value > 0:                    
                    total_grams += var_value * int(j[:-1])
    print(f"{i} = {total_grams} grams")